In [16]:
import numpy as np
import pandas as pd
from sko.ACA import ACA_TSP
from utils import parse_UK_Data

dataset_path = "../datasets/UK10_1.txt"

meta_data, distance_data, station_data = parse_UK_Data(dataset_path)

In [17]:
meta_data

,Customer Amount,Vehicle Curb Weight(kg),Max Load(kg),Minimum Speed(km/h),Maximum Speed(km/h)
0,10,6350,3650,20,90


In [18]:
distance_data

,0,1,2,3,4,5,6,7,8,9,10
0,0,41150,25680,54200,95380,15910,88960,74120,26010,88181,66070
1,40660,0,51980,32800,99870,42210,75660,63880,24350,72070,26250
2,25010,51780,0,61520,74050,12890,69270,52590,42910,73400,76700
3,54270,32750,61560,0,77030,51930,42930,31920,49480,39500,29500
4,94930,100030,74070,76930,0,81260,55600,46100,111960,61700,106350
5,15830,42600,12880,52340,81050,0,78000,61320,33730,82130,67520
6,88751,75700,69300,43030,55210,78040,0,17200,90550,6520,68800
7,73340,63440,52480,31830,46430,61220,17130,0,75520,21260,61250
8,25990,24350,43780,49530,111730,34010,90550,75740,0,88960,48920
9,88411,71740,73420,39430,61390,82160,6550,21320,88830,0,64010


In [19]:
station_data

,City Name,Demand(kg),Ready Time(sec),Due Time(sec),Service Time(sec)
0,Kingston_upon_Hull,0,0,32400,0
1,Pocklington,721,2171,22139,1442
2,Brough,814,644,21053,1628
3,Selby,620,1049,20424,1240
4,Boughton,311,5119,24241,622
5,Barton_upon_Humber,167,2338,21697,334
6,Darfield,513,3675,24120,1026
7,Bentley,568,4401,25046,1136
8,Watton,763,3485,24335,1526
9,Cudworth,558,3740,24049,1116


In [ ]:
num_points = station_data.shape[0]

